In [1]:
# Import dependencies and configure gmaps

import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
import json
from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
# Read in weather data csv

weather_pd = pd.read_csv("../Weatherpy/weatherAPI_data.csv")
weather_pd.head()

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,Oyama,1853483,JP,36.30,139.80,56.12,71,20,1.12
1,Ushuaia,3833367,AR,-54.80,-68.30,51.80,50,40,4.70
2,Veldhoven,2745706,NL,51.42,5.40,51.24,87,90,8.05
3,Hilo,5855927,US,19.73,-155.09,77.99,69,40,4.70
4,Mataura,6201424,NZ,-46.19,168.86,46.00,96,100,1.01


In [4]:
# Create gmap figure then add heatmap layer using city lat/lng's for locations and city humidity for the weight

cities = weather_pd[['Latitude', 'Longitude']]
weights = weather_pd['Humidity']
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
humidity_map = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))
humidity_map.add_layer(gmaps.heatmap_layer(cities, weights=weights, max_intensity=100, dissipating=False))
humidity_map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [25]:
# Use loc function to create dataframe of cities with my ideal weather. I choose cool, cloudless and low humidity to try to 
# find cities that have crisp fall weather

ideal_cond = weather_pd.loc[(weather_pd['Temperature'] >= 60) & (weather_pd['Temperature'] < 70) & (weather_pd['Humidity'] < 60) & (weather_pd['Cloudiness'] < 20)]
ideal_cond

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
26,Copiapó,3893656,CL,-27.37,-70.33,69.80,49,0,12.75
45,Rawson,3839307,AR,-43.30,-65.10,68.20,24,0,23.40
50,Lachhmangarh,1271910,IN,27.82,75.03,68.83,27,3,2.33
66,Mamedkala,529679,RU,42.17,48.12,64.96,40,2,3.74
82,Constitución,3893726,CL,-35.33,-72.42,60.15,53,0,18.07
253,Awjilah,88533,LY,29.11,21.29,65.91,48,0,3.36
260,Mīnāb,123941,IR,27.15,57.08,69.80,37,0,6.93
273,Khairābād,1267175,IN,27.53,80.75,69.24,31,0,4.68
300,Talcahuano,3870282,CL,-36.72,-73.12,60.80,48,0,14.99
321,Pahrump,5509851,US,36.21,-115.98,63.57,34,0,3.00


In [26]:
# Create empty list to hold hotel names

hotel_list = []


# set up a parameters dictionary
params = {
    "keyword": 'Hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": gkey
     }

# Iterate through the data frame rows in order to use city's lat/lng to request hotel names

for index, row in ideal_cond.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    hotels = response.json()

# Append hotel names to empty list if found    
    
    try:

        hotel_list.append(hotels['results'][1]['name'])

# Handle exceptions, append 'hotel not found' to list to keep hotels list same length as ideal_cond dataframe        
        
    except (KeyError, IndexError):
        hotel_list.append('Hotel not found')
        print('No hotel found, skipping city')


# Used the code below to figure out how to find the hotel name specifically
# print(json.dumps(hotels, indent=8, sort_keys=True))



No hotel found, skipping city
No hotel found, skipping city
No hotel found, skipping city
No hotel found, skipping city


In [27]:
# Set the hotel list as a new column in dataframe

ideal_cond['Hotel'] = hotel_list
ideal_cond

C:\Users\hanna\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel
26,Copiapó,3893656,CL,-27.37,-70.33,69.80,49,0,12.75,Hotel Chagall
45,Rawson,3839307,AR,-43.30,-65.10,68.20,24,0,23.40,Casa de mar
50,Lachhmangarh,1271910,IN,27.82,75.03,68.83,27,3,2.33,Hotel Aashish Resort
66,Mamedkala,529679,RU,42.17,48.12,64.96,40,2,3.74,Hotel not found
82,Constitución,3893726,CL,-35.33,-72.42,60.15,53,0,18.07,Bed & Breakfast Sweet Dreams
253,Awjilah,88533,LY,29.11,21.29,65.91,48,0,3.36,Hotel not found
260,Mīnāb,123941,IR,27.15,57.08,69.80,37,0,6.93,هتل صدف
273,Khairābād,1267175,IN,27.53,80.75,69.24,31,0,4.68,Hotel Apple
300,Talcahuano,3870282,CL,-36.72,-73.12,60.80,48,0,14.99,Hotel Düsseldorf
321,Pahrump,5509851,US,36.21,-115.98,63.57,34,0,3.00,Holiday Inn Express & Suites Pahrump


In [40]:
# Use iloc to pull out rows with hotel names in English

final_df = ideal_cond.iloc[[0,1,2,4,7,8,9]] 
final_df

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel
26,Copiapó,3893656,CL,-27.37,-70.33,69.80,49,0,12.75,Hotel Chagall
45,Rawson,3839307,AR,-43.30,-65.10,68.20,24,0,23.40,Casa de mar
50,Lachhmangarh,1271910,IN,27.82,75.03,68.83,27,3,2.33,Hotel Aashish Resort
82,Constitución,3893726,CL,-35.33,-72.42,60.15,53,0,18.07,Bed & Breakfast Sweet Dreams
273,Khairābād,1267175,IN,27.53,80.75,69.24,31,0,4.68,Hotel Apple
300,Talcahuano,3870282,CL,-36.72,-73.12,60.80,48,0,14.99,Hotel Düsseldorf
321,Pahrump,5509851,US,36.21,-115.98,63.57,34,0,3.00,Holiday Inn Express & Suites Pahrump


In [50]:
# Create list of lat/lng coordinates

marker_locations = [
    (final_df.iloc[0,[3,4]]),
    (final_df.iloc[1,[3,4]]),
    (final_df.iloc[2,[3,4]]),
    (final_df.iloc[3,[3,4]]),
    (final_df.iloc[4,[3,4]]),
    (final_df.iloc[5,[3,4]]),
    (final_df.iloc[6,[3,4]])
]

# Create list of dicts to hold info box pop up text

ideal_cities_info = [
    {'Hotel': final_df.iloc[0,9], 'City': final_df.iloc[0,0], 'Country': final_df.iloc[0,2]},
    {'Hotel': final_df.iloc[1,9], 'City': final_df.iloc[1,0], 'Country': final_df.iloc[1,2]},
    {'Hotel': final_df.iloc[2,9], 'City': final_df.iloc[2,0], 'Country': final_df.iloc[2,2]},
    {'Hotel': final_df.iloc[3,9], 'City': final_df.iloc[3,0], 'Country': final_df.iloc[3,2]},
    {'Hotel': final_df.iloc[4,9], 'City': final_df.iloc[4,0], 'Country': final_df.iloc[4,2]},
    {'Hotel': final_df.iloc[5,9], 'City': final_df.iloc[5,0], 'Country': final_df.iloc[5,2]},
    {'Hotel': final_df.iloc[6,9], 'City': final_df.iloc[6,0], 'Country': final_df.iloc[6,2]}
]

# Used provided info box template, changed out names to fit my data

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**city) for city in ideal_cities_info]

# Added markers as layer to my heatmap

markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
humidity_map.add_layer(markers)
humidity_map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…